# DS-GA-3001 Advanced Python for Data Science

Before you turn this problem in, make sure you **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart). You can then run the cells **in order**, during the class.

Any textual answers that need to be provided will be marked with "YOUR ANSWER HERE". Replace this text with your answer to the question.

Any code answers that need to be provided will be marked with:

```
# YOUR CODE HERE
raise NotImplementedError()
```

Replace all this code with your answer to the question. If you do not answer the question, the `NotImplementedError` exception will be raised, which will indicate to the grader that no answer has been supplied.

In many cases, code answers will also have some associated test code. You should execute the tests after you have entered your code in order to ensure that your answer is correct. You should not proceed to the next question until your answer is correct.

Finally, insert your Net ID and the Net ID's of any collaborators in the cell below.

In [ ]:
NET_ID = "hz990"
COLLABORATORS = ""

---

# Introduction to GPU Computing

This module provides an introduction to using Python for GPU computing. It is split into two parts. The first part provides an overview of the NVIDIA GPU architecture and the CUDA programming model. The second part will provide hands on experience using `NumbaPro` to develope GPU programs using Python.

## GPU Architercture

A central processing unit (CPU) is designed to handle complex tasks, such as time slicing, virtual machine emulation, complex control flows and branching, security, etc. In contrast, graphical processing unites (GPUs) only do one thing well. They handle billions of repetitive low level tasks. Originally designed for the rendering of triangles in 3D graphics, they have thousands of arithmetic logic units (ALUs) compared with traditional CPUs that commonly have only 4 or 8. Many types of scientific algorithms spend most of their time doing just what GPUs are good for: performing billions of repetitive arithmetic operations. Computer scientists have been quick to harness the power of GPUs for computational science applications.

The following diagram shows how GPU performance has increased compared to traditional CPU architetures.

![](http://docs.nvidia.com/cuda/cuda-c-programming-guide/graphics/floating-point-operations-per-second.png)

The reason behind the discrepancy in floating-point capability between the CPU and the GPU is that the GPU is specialized for compute-intensive, highly parallel computation - exactly what graphics rendering is about - and therefore designed such that more transistors are devoted to data processing rather than data caching and flow control.

![](http://www.frontiersin.org/files/Articles/70265/fgene-04-00266-HTML/image_m/fgene-04-00266-g001.jpg)

More specifically, the GPU is especially well-suited to address problems that can be expressed as **data-parallel computations** - the same program is executed on many data elements in parallel - with a high ratio of arithmetic operations to memory operations. 

Because the same program is executed for each data element, there is a lower requirement for sophisticated flow control, and because it is executed on many data elements and has high arithmetic intensity, the memory access latency can be hidden with calculations instead of big data caches.

### Data-parallel processing maps data elements to parallel processing threads

Many applications that process large data sets can use a data-parallel programming model to speed up the computations. In 3D rendering, large sets of pixels and vertices are mapped to parallel threads. Similarly, image and media processing applications such as post-processing of rendered images, video encoding and decoding, image scaling, stereo vision, and pattern recognition can map image blocks and pixels to parallel processing threads. In fact, many algorithms outside the field of image rendering and processing are accelerated by data-parallel processing, from general signal processing or physics simulation to computational finance or computational biology.

The advent of multicore CPUs and manycore GPUs means that mainstream processor chips are now parallel systems. Furthermore, their parallelism continues to scale with Moore's law. The challenge is to develop application software that transparently scales its parallelism to leverage the increasing number of processor cores, much as 3D graphics applications transparently scale their parallelism to manycore GPUs with widely varying numbers of cores.

![](http://www.nvidia.com/docs/IO/143716/cpu-and-gpu.jpg)

##### from IPython.display import YouTubeVideo
YouTubeVideo('-P28LKWTzrI')

## GPU Programming

When computer scientists first attempted to use GPUs for scientific computing, the scientific codes had to be mapped onto the matrix operations for manipulating traingles. This was incredibly difficult to do, and took a lot of time and dedication. However, there are now high level languages (such as CUDA and OpenCL) that target the GPUs directly, so GPU programming is rapidly becoming mainstream in the scientific community.

A GPU program comprises two parts: a ***host part*** the runs on the CPU and one or more ***kernels*** that run on the GPU. Typically, the CPU portion of the program is used to set up the parameters and data for the computation, while the kernel portion performs the actual computation. In some cases the CPU portion may comprise a parallel program that performs message passing operations using MPI.

![](http://www.nvidia.com/docs/IO/143716/how-gpu-acceleration-works.png)

### CUDA

In November 2006, NVIDIA introduced **CUDA**, which originally stood for "Compute Unified Device Architecture", a general purpose parallel computing platform and programming model that leverages the parallel compute engine in NVIDIA GPUs to solve many complex computational problems in a more efficient way than on a CPU. 

The CUDA parallel programming model has three key abstractions at its core:
- a hierarchy of thread groups
- shared memories
- barrier synchronization

There are exposed to the programmer as a minimal set of language extensions.

<div class="alert alert-success">
In parallel programming, granularity means the amount of computation in relation to communication (or transfer) of data.

Fine-grained parallelism means individual tasks are relatively small in terms of code size and execution time. The data is transferred among processors frequently in amounts of one or a few memory words. Coarse-grained is the opposite in that data is communicated infrequently, after larger amounts of computation
</div>

The CUDA abstractions provide fine-grained data parallelism and thread parallelism, nested within coarse-grained data parallelism and task parallelism. They guide the programmer to partition the problem into coarse sub-problems that can be solved independently in parallel by blocks of threads, and each sub-problem into finer pieces that can be solved cooperatively in parallel by all threads within the block.

A kernel is executed in parallel by an array of threads.
- All threads run the same code.
- Each thread has an ID that it uses to compute memory addresses and make control decisions.

![](http://drive.google.com/uc?export=view&id=0B_3lImS7uRMgS0dVNS1zUTRXNHc)

Threads are arranged as a grid of thread blocks.
- Different kernels can have different grid/block configuration
- Threads from the same block have access to a shared memory and their execution can be synchronized

![](http://drive.google.com/uc?export=view&id=0B_3lImS7uRMgREFjZXZQNUVPVGM)

Thread blocks are required to execute independently: It must be possible to execute them in any order, in parallel or in series. This independence requirement allows thread blocks to be scheduled in any order across any number of cores, enabling programmers to write code that scales with the number of cores. Threads within a block can cooperate by sharing data through some shared memory and by synchronizing their execution to coordinate memory accesses.

The grid of blocks can be 1 or 2-dimentional and the thread blocks can be 1, 2, or 3-dimensional

![](http://drive.google.com/uc?export=view&id=0B_3lImS7uRMgdTVhYndTNWd3V00)

The CUDA architecture is built around a scalable array of multithreaded ***Streaming Multiprocessors (SMs)*** as shown below. Each SM has a set of execution units, a set of registers and a chunk of shared memory. 

In an NVIDIA GPU, the basic unit of execution is the ***warp***. A warp is a collection of threads, 32 in current implementations, that are executed simultaneously by an SM. Multiple warps can be executed on an SM at once.

When a CUDA program on the host CPU invokes a kernel grid, the blocks of the grid are enumerated and distributed to SMs with available execution capacity. The threads of a thread block execute concurrently on one SM, and multiple thread blocks can execute concurrently on one SM. As thread blocks terminate, new blocks are launched on the vacated SMs.

The mapping between warps and thread blocks *can* affect the performance of the kernel. It is usually a good idea to keep the size of a thread block a multiple of 32 in order to avoid this as much as possible.

![](https://devblogs.nvidia.com/parallelforall/wp-content/uploads/2016/04/gp100_SM_diagram-624x452.png)

### Thread Identity

The index of a thread and its $thread\ ID$ relate to each other as follows:
- For a 1-dimensional block, the thread index and $thread\ ID$ are the same
- For a 2-dimensional block, the thread index $(x, y)$ has $thread\ ID = x + y D_x$, for block size $(D_x, D_y)$
- For a 3-dimensional block, the thread index $(x, y, x)$ has $thread\ ID = x + y D_x + z D_x D_y$, for block size $(D_x, D_y, D_z)$

When a kernel is started, the number of blocks per grid and the number of threads per block are fixed (`gridDim` and `blockDim`). CUDA makes four pieces of information available to each thread:
- The thread index (`threadIdx`)
- The block index (`blockIdx`)
- The size and shape of a block (`blockDim`)
- The size and shape of a grid (`gridDim`)

Typically, each thread in a kernel will compute one element of an array. There is a common pattern to do this that most CUDA programs use:

For a 1-dimensional grid:

In [ ]:
tx = cuda.threadIdx.x
bx = cuda.blockIdx.x
bw = cuda.blockDim.x
i = tx + bx * bw
array[i] = compute(i)

For a 2-dimensional grid:

In [ ]:
tx = cuda.threadIdx.x
ty = cuda.threadIdx.y
bx = cuda.blockIdx.x
by = cuda.blockIdx.y
bw = cuda.blockDim.x
bh = cuda.blockDim.y
x = tx + bx * bw
y = ty + by * bh
array[x, y] = compute(x, y)

### Memory Hierarchy

The CPU and GPU have separate ***memory spaces***. This means that data that is processed by the GPU must be moved from the CPU to the GPU before the computation starts, and the results of the computation must be moved back to the CPU once processing has completed.

#### Global memory

This memory is accessible to all threads as well as the host (CPU).
- Global memory is allocated and deallocated by the host
- Used to initialize the data that the GPU will work on

![](http://drive.google.com/uc?export=view&id=0B_3lImS7uRMgUjVndThBaU52ejg)

#### Shared memory

Each ***thread block*** has its own shared memory
- Accessible only by threads within the block
- Much faster than local or global memory
- Requires special handling to get maximum performance
- Only exists for the lifetime of the block

![](http://drive.google.com/uc?export=view&id=0B_3lImS7uRMgLV94VUJkYW1qV2s)

#### Local memory

Each ***thread*** has its own private local memory
- Only exists for the lifetime of the thread
- Generally handled automatically by the compiler

![](http://drive.google.com/uc?export=view&id=0B_3lImS7uRMgV0Eybks2cXlCdVU)

#### Constant and texture memory

These are read-only memory spaces accessible by all threads.
- Constant memory is used to cache values that are shared by all functional units
- Texture memory is optimized for texturing operations provided by the hardware

# Continued next week: Python on GPUs with NumbaPro